In [10]:
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict
from transformers import Trainer, TrainingArguments, pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('hs_top100.csv')
df.head()

,row_id,Row Number,Tweet Treated,Label
0,fc459cd439c7fc8ada3b42ab80090747fa55024234516e...,19708,not a fucking many as zelensky for being a w...,Hate
1,ffa430452f43a76b3a25eef39b51ca7a0d9051ae23e0db...,19981,on the brink of nuclear war and she droppin mi...,Hate
2,f6809f8d0fc5511a9f715c167757d46b21fda05008f6a7...,19241,"fuck zelensky, fuck ukraine, fuck our governme...",Hate
3,f67aa5365c8144d61f39ccdd6333669fd1b69de41ccba0...,19238,i support the ukrainians to resist to the end...,Hate
4,f8d080e732b313c3f20792453e3a8ee91e32c6925c806d...,19438,no one cares about your shitty russian revis...,Hate


In [4]:
df["hate"] = df["Label"].apply(lambda x: 1 if x == "Hate" else 0)
df["text"] = df["Tweet Treated"]

dfFinal = df[["text", "hate"]]

In [21]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class TextDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.encodings = tokenizer(data['text'].tolist(),
                                    truncation=True,
                                    padding=True,
                                    max_length=max_length,
                                    return_tensors='pt')
        self.labels = torch.tensor(data['label'].tolist())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [14]:
dfFinal

,text,hate,train_test_split,label
0,not a fucking many as zelensky for being a w...,1,train,1
1,on the brink of nuclear war and she droppin mi...,1,test,1
2,"fuck zelensky, fuck ukraine, fuck our governme...",1,test,1
3,i support the ukrainians to resist to the end...,1,test,1
4,no one cares about your shitty russian revis...,1,train,1
...,...,...,...,...
95,"really thought it was ukraine footage, would ...",0,train,0
96,this isn't appeasement but it would still be d...,0,test,0
97,group of cowards and desperates forgot the rul...,0,train,0
98,damn. happy gilmore can’t go to russia 🥲,0,train,0


In [15]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

# Example function to preprocess data
def preprocess_data(data):
    encodings = tokenizer(data['text'].tolist(),
                          truncation=True,
                          padding=True,
                          max_length=256,
                          return_tensors='pt')
    labels = torch.tensor(data['label'])
    return {'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask'],
            'labels': labels}

# Apply preprocessing
preprocessed_data = preprocess_data(dfFinal)


In [24]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Load pre-trained model and tokenizer
model_name = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Prepare dataset
train_df, test_df = train_test_split(dfFinal, test_size=0.3, random_state=42)

train_dataset = TextDataset(train_df, tokenizer)
test_dataset = TextDataset(test_df, tokenizer)

# Create data loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(20):  # Number of epochs
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')

    model.eval()
    with torch.no_grad():
        total_correct = 0
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted = torch.max(logits, dim=1)
            total_correct += (predicted == labels).sum().item()

        accuracy = total_correct / len(test_loader.dataset)
        print(f'Test Accuracy: {accuracy:.4f}')

# Save the model
# model.save_pretrained('./modernbert_text_classification')
# tokenizer.save_pretrained('./modernbert_text_classification')


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Alexandre\AppData\Local\Temp\ipykernel_11136\1398806696.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\Alexandre\AppData\Local\Temp\ipykernel_11136\1398806696.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch 1, Loss: 0.7675955653190613
Test Accuracy: 0.5333
Epoch 2, Loss: 0.5927321672439575
Test Accuracy: 0.5333
Epoch 3, Loss: 0.5150217473506927
Test Accuracy: 0.5333
Epoch 4, Loss: 0.4453367114067078
Test Accuracy: 0.5667
Epoch 5, Loss: 0.38869022130966185
Test Accuracy: 0.5000
Epoch 6, Loss: 0.3123825669288635
Test Accuracy: 0.5667
Epoch 7, Loss: 0.21291923224925996
Test Accuracy: 0.6000
Epoch 8, Loss: 0.1401660770177841
Test Accuracy: 0.6333
Epoch 9, Loss: 0.06456725671887398
Test Accuracy: 0.6333
Epoch 10, Loss: 0.03076095748692751
Test Accuracy: 0.6333
Epoch 11, Loss: 0.01270284727215767
Test Accuracy: 0.6333
Epoch 12, Loss: 0.0054933502804487945
Test Accuracy: 0.6333
Epoch 13, Loss: 0.0029982195468619467
Test Accuracy: 0.6333


KeyboardInterrupt: 